# **Computer**-Vision-II-Air-Drums

github: https://github.com/ric31303/Computer-Vision-II-Air-Drums.git

# Set Env

In [0]:
!git clone https://github.com/ric31303/Computer-Vision-II-Air-Drums.git --branch dev

%ls

Cloning into 'Computer-Vision-II-Air-Drums'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 84.41 MiB | 31.05 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Computer-Vision-II-Air-Drums/  sample_data/


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Run LRCN model

In [0]:
%run Computer-Vision-II-Air-Drums/LRCN.py
# import pandas as pd

import os
# url = 'sample_data/california_housing_test.csv'
# df1 = pd.read_csv(url)
# print(df1)
path = 'drive/My Drive/ML_DATA/input/'
# entries = os.listdir(path)
# print(entries)
lrcn_model = LRCN(path) 


Run LRCN


# Build LRCN

In [0]:

# modify model parameters
lrcn_model.batch_size = 16
lrcn_model.epoch_num = 1
lrcn_model.class_weight = {0: 1.,
        1: 3.,
        2: 3.,
        3: 3.
        }
# build model

vgg = VGG16(
  include_top=False,
    weights='imagenet',
    input_shape=lrcn_model.input_data_shape[1:]
)
model = Sequential()
model.add(
    TimeDistributed(vgg, input_shape= lrcn_model.input_data_shape)
)
model.add(
    TimeDistributed(
        Flatten()
    )
)
model.add(LSTM(256, activation='relu', return_sequences=False))
# finalize with standard Dense, Dropout...
model.add(Dense(64, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(lrcn_model.output_size, activation='softmax'))

model.save(MODEL_PATH)
model.compile('adam', loss='categorical_crossentropy',metrics=["accuracy"])

print(model.summary())

# callbacks: early_stop, check_point
es_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=2)
cp_callback = keras.callbacks.ModelCheckpoint(filepath=CP_PATH,save_weights_only=True,verbose=1)

(trainX, testX, trainY, testY) = lrcn_model.getData()
# train the network
print("[INFO] training network...")

trainX = np.array(trainX)
trainY = np.array(trainY)
testX = np.array(testX)
testY = np.array(testY)

H = model.fit(trainX, trainY, validation_data=(testX, testY),
batch_size=lrcn_model.batch_size, epochs=lrcn_model.epoch_num,
class_weight=lrcn_model.class_weight,callbacks=[es_callback,cp_callback], verbose=1)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=lrcn_model.batch_size)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=lrcn_model.targets))


In [0]:
loss = H.history["loss"]
val_loss = H.history["val_loss"]
accuracy = H.history["accuracy"]
val_accuracy = H.history["val_accuracy"]
print(len(loss))

training_number = len(loss)
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, training_number), loss, label="train_loss")
plt.plot(np.arange(0, training_number), val_loss, label="val_loss")
plt.plot(np.arange(0, training_number), accuracy, label="train_acc")
plt.plot(np.arange(0, training_number), val_accuracy, label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()